In [17]:
import os
os.chdir("../")

In [19]:
%pwd


'/Users/sudipaguha007/Documents/RSH'

In [21]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [23]:
def load_pdf_file(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=pyPDFLoader)
    documents=loader.load()
    return Documents

In [25]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)  # Corrected class name
    documents = loader.load()
    return documents  # Corrected return variable name

extracted_data = load_pdf_file(data='Data/')



In [26]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [29]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 212


In [31]:
from langchain.embeddings import HuggingFaceEmbeddings
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings
embeddings = download_hugging_face_embeddings()

/var/folders/1j/j3w677k563jg8rnd95q4z9vr0000gn/T/ipykernel_32325/921514773.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/opt/anaconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [33]:
query_result=embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


In [35]:
#query_result

In [37]:
from dotenv import load_dotenv
load_dotenv()

True

In [39]:
import os
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')


In [ ]:

from pinecone import Pinecone, ServerlessSpec
import os

pc= Pinecone(api_key=PINECONE_API_KEY)

index_name = "rshbot"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

    

In [43]:
import os
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY


In [45]:
from langchain_pinecone import PineconeVectorStore
docsearch=PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [73]:
from langchain_pinecone import PineconeVectorStore
docsearch=PineconeVectorStore.from_existing_index(
    
    index_name=index_name,
    embedding=embeddings,
)

In [47]:
docsearch

In [49]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})


In [51]:
retrieved_docs = retriever.invoke("What is Rain water harvesting?")


In [53]:
retrieved_docs


[Document(id='aeb3a401-8749-4aaf-882d-ea2324beb9cc', metadata={'page': 7.0, 'source': 'Data/rain_water_harvesting.pdf'}, page_content='Rain water harvesting can be effectively implemented in our'),
 Document(id='9926b8aa-6ddc-4b7e-968b-d30f529554f3', metadata={'page': 7.0, 'source': 'Data/rain_water_harvesting.pdf'}, page_content='Rain water harvesting can be effectively implemented in our'),
 Document(id='bb776100-3e12-442a-9339-f02aceea32f7', metadata={'page': 6.0, 'source': 'Data/rain_water_harvesting.pdf'}, page_content='table has already gone down abnormally and if we do not wake up even now then our future generations may have to face severe crisis of water. The rains are important source of water and if we can harvest rain water, the scarcity of water can be eliminated altogether. Therefore, it is our bounden duty to conserve the rain water in the form of rain water harvesting.\nThe book on “Rain Water Harvesting” is an attempt by')]

In [59]:
from langchain_mistralai import ChatMistralAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
import os


load_dotenv()

mistral_api_key = os.getenv("MISTRAL_API_KEY")


llm = ChatMistralAI(api_key= mistral_api_key, model="mistral-tiny")  


system_prompt = (
    "You are an assistant for question-answering tasks."
    " Use the following retrieved context to answer the question."
    " If you don't know the answer, say so."
    " Keep your answer concise (max 3 sentences)."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])


question_answer_chain = create_stuff_documents_chain(llm, prompt)


rag_chain = create_retrieval_chain(retriever, question_answer_chain)
response = rag_chain.invoke({"input": "What is Rainwater Harvesting?"})
print(response["answer"])


Rainwater harvesting is the process of collecting and storing rainwater from rooftops, impermeable surfaces, or other runoff areas for later use. It can be an effective way to manage water resources during periods of drought or to reduce reliance on municipal water supplies.


In [63]:
response = rag_chain.invoke({"input": "Steps for Rain water harvesting?"})
print(response["answer"])

Rain water harvesting can be effectively implemented by following these steps: 1. Identify a suitable location for the rainwater harvesting structure. 2. Design the system considering the roof area, storage capacity, and outlet requirements. 3. Construct the system using materials like concrete, plastic, or steel, and ensure proper installation and maintenance.


In [67]:
response = rag_chain.invoke({"input": "tell me more about rain water harvesting?"})
print(response["answer"])

Rain water harvesting is the process of collecting and storing rainwater from rooftops, pavements, or other surfaces for later use. This can be done using various methods such as roof top rainwater harvesting systems, percolation pits, and permeable surfaces. The collected water can be used for non-potable purposes like irrigation, flushing, and toilet use, reducing the dependence on groundwater or other sources.


In [73]:
response = rag_chain.invoke({"input": "i have a small house.How can I implement rainwater harvesting?"})
print(response["answer"])

For a small house, you can implement rainwater harvesting by installing a simple rooftop rainwater harvesting system. This typically involves installing a gutter system to collect rainwater, a filter to remove debris, and a storage tank. You may also need a pump if your storage tank is located above your usage point. Consult local regulations and consider hiring a professional for installation.
